## Wordle
words from https://github.com/dwyl/english-words    
letter freqs from #https://www3.nd.edu/~busiforc/handouts/cryptography/letterfrequencies.html

In [1]:
import numpy as np
import random 
import json

from wordleoracle import WordleOracle ,get_possible_words

In [2]:
Orc = WordleOracle()

In [3]:
🟨⬛🟩
print("\U0001F642")
print('\u0420\u043e\u0441\u0441\u0438\u044f')
print('\U+1F7E8')

SyntaxError: invalid character '🟨' (U+1F7E8) (<ipython-input-3-218f8271774b>, line 1)

In [4]:
with open('letter_freq.json') as json_file:
    letter_freq_dict = json.load(json_file)

In [5]:
def get_new_possible_words(previous_possible_words,guess_list= [],results_list= []):
    possible_words = previous_possible_words.copy()
    last_result = results_list[-1]
    last_guess = guess_list[-1]
    for i in range(len(last_result)):
        c = last_guess[i] 
        #print(c,"was",last_result[i])
        if last_result[i] == 0:
            new_possible_words = [word for word in possible_words if c not in word]
            possible_words = new_possible_words
        if last_result[i] == 1:
            new_possible_words = [word for word in possible_words if c in word]
            possible_words = new_possible_words
        if last_result[i] == 2:
            new_possible_words = [word for word in possible_words if word[i] == c]
            possible_words = new_possible_words
    return possible_words
    
def make_guess_random(possible_words,guess_list,result_list):
    return random.choice(possible_words)

def test_guess(guess):
    secret_word = "crunk"
    if not isinstance(guess, str):
        raise ValueError(guess,"not a string")
    if not len(guess)==5:
        raise ValueError(guess,"not 5 letter word")
    results = np.zeros((5))
    guess = guess.lower()
    for i in range(5): #  not optimised for speed
        if guess[i] == secret_word[i]:
            results[i]=2
        elif guess[i] in secret_word:
            results[i] = 1
#         else: 
#           results[i] = 0
    return results

## Basic random guessing

In [6]:
possible_words = get_possible_words()
guess_list = []
result_list = []
finished = False
i = 0
while(i<6 and finished==False):
    guess = random.choice(possible_words)
    result = test_guess(guess)
    guess_list.append(guess)
    result_list.append(result)
    print("round",i,"| guess was",guess,"| result == ",result)
    print(len(possible_words))
    if (result == 2).all():
        print("Correct!")
        finished = True
    i=i+1
if not finished:
    print("Failed :(")

round 0 | guess was agend | result ==  [0. 0. 0. 2. 0.]
15918
round 1 | guess was ahush | result ==  [0. 0. 2. 0. 0.]
15918
round 2 | guess was hiera | result ==  [0. 0. 0. 1. 0.]
15918
round 3 | guess was boodh | result ==  [0. 0. 0. 0. 0.]
15918
round 4 | guess was piast | result ==  [0. 0. 0. 0. 0.]
15918
round 5 | guess was liszt | result ==  [0. 0. 0. 0. 0.]
15918
Failed :(


## Random Guessing (pruning)

In [6]:
rounds = 3
player_results = np.zeros(rounds)
for k in range(rounds):
    Orc.play()
    possible_words = get_possible_words()
    guess_list = []
    result_list = []
    finished = False
    i = 0
    while(i<27 and not finished):
        guess = random.choice(possible_words)
        result = Orc.test_guess(guess)
        guess_list.append(guess)
        result_list.append(result)
        new_possible_words = get_new_possible_words(possible_words,guess_list,result_list)
        possible_words = new_possible_words
        print("round",i,"| guess was",guess,"| possible words remaining",len(possible_words),"result == ",result)
        if (result == 2).all():
            player_results[k]=i
            print("Correct!")
            finished = True
        i=i+1
    if not finished: # impossible to reach
        player_results[k]= np.inf
        print("Failed :( , word was ",Orc.get_word())
print("finished, average number of rounds was",np.mean(player_results))

round 0 | guess was colat | possible words remaining 2736 result ==  [0. 0. 0. 1. 0.]
round 1 | guess was naira | possible words remaining 239 result ==  [1. 2. 0. 0. 1.]
round 2 | guess was mayan | possible words remaining 149 result ==  [0. 2. 0. 1. 1.]
round 3 | guess was fauns | possible words remaining 28 result ==  [0. 2. 0. 1. 2.]
round 4 | guess was vanes | possible words remaining 14 result ==  [0. 2. 2. 0. 2.]
round 5 | guess was bangs | possible words remaining 1 result ==  [2. 2. 2. 2. 2.]
Correct!
round 0 | guess was twilt | possible words remaining 2114 result ==  [1. 0. 0. 0. 1.]
round 1 | guess was turco | possible words remaining 313 result ==  [1. 0. 0. 0. 1.]
round 2 | guess was tempo | possible words remaining 15 result ==  [1. 1. 0. 1. 1.]
round 3 | guess was epopt | possible words remaining 3 result ==  [2. 1. 1. 1. 1.]
round 4 | guess was exopt | possible words remaining 2 result ==  [2. 0. 1. 1. 1.]
round 5 | guess was estop | possible words remaining 1 result =

## Random Guessing (crane)

In [7]:
rounds = 100
player_results = np.zeros(rounds)
for k in range(rounds):
    Orc.play()
    possible_words = get_possible_words()
    guess_list = []
    result_list = []
    finished = False
    i = 0
    while(i<27 and not finished):
        if i ==0:
            guess = "crane"
        else:
            guess = random.choice(possible_words)
        result = Orc.test_guess(guess)
        guess_list.append(guess)
        result_list.append(result)
        new_possible_words = get_new_possible_words(possible_words,guess_list,result_list)
        possible_words = new_possible_words
        #print("round",i,"| guess was",guess,"| possible words remaining",len(possible_words),"result == ",result)
        if (result == 2).all():
            player_results[k]=i
            #print("Correct!")
            finished = True
        i=i+1
    if not finished: # impossible to reach
        player_results[k]= np.inf
        #print("Failed :( , word was ",Orc.get_word())
print("finished, average number of rounds was",np.mean(player_results))

finished, average number of rounds was 5.307


## weighted guess (too slow doesn't work)

In [13]:
def make_guess_weighted(possible_words): #  too slow
    weights=[]
    for choice in possible_words:
        weight = 0
        for letter, freq in letter_freq_dict.items():
            if letter in choice:
                weight+= freq
        weights.append(weight) 
        weights = [weight/ sum(weights) for weight in weights]

    guesses = random.choices(population = possible_words,weights=weights)

rounds = 1
player_results = np.zeros(rounds)
for k in range(rounds):
    Orc.play()
    possible_words = get_possible_words()
    guess_list = []
    result_list = []
    finished = False
    i = 0
    while(i<27 and not finished):
        guess = make_guess_weighted(possible_words)
        result = Orc.test_guess(guess)
        guess_list.append(guess)
        result_list.append(result)
        new_possible_words = get_new_possible_words(possible_words,guess_list,result_list)
        possible_words = new_possible_words
        #print("round",i,"| guess was",guess,"| possible words remaining",len(possible_words),"result == ",result)
        if (result == 2).all():
            player_results[k]=i
            #print("Correct!")
            finished = True
        i=i+1
    if not finished: # impossible to reach
        player_results[k]= np.inf
        #print("Failed :( , word was ",Orc.get_word())
print("finished, average number of rounds was",np.mean(player_results))

KeyboardInterrupt: 